In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os

from tqdm import tqdm

import torch
from torch import nn
from torch.nn import functional as F
import torchvision

import torchvision.models as models


""" !wget "https://raw.githubusercontent.com/d2l-ai/d2l-en/master/d2l/torch.py" -q -O d2l.py
import d2l

import keras """


' !wget "https://raw.githubusercontent.com/d2l-ai/d2l-en/master/d2l/torch.py" -q -O d2l.py\nimport d2l\n\nimport keras '

# The dataset are sets of images of drivers, carrying out different actions.
### The 10 classes to predict are:
c0: normal driving \
c1: texting - right \
c2: talking on the phone - right \
c3: texting - left \
c4: talking on the phone - left \
c5: operating the radio \
c6: drinking \
c7: reaching behind \
c8: hair and makeup \
c9: talking to passenger 

In [35]:
df = pd.read_csv("src/driver_imgs_list.csv")

# For pilot study, we approach this with binary classification: 
# Normal driving c0 vs not_c0 (designated as c1) 
#df["classname"].loc[df["classname"] != "c0"] = "c1"
print(df["subject"].unique(), len(df["subject"].unique()))

np.unique(df["classname"], return_counts=True)

people = (df["subject"].unique()

['p002' 'p012' 'p014' 'p015' 'p016' 'p021' 'p022' 'p024' 'p026' 'p035'
 'p039' 'p041' 'p042' 'p045' 'p047' 'p049' 'p050' 'p051' 'p052' 'p056'
 'p061' 'p064' 'p066' 'p072' 'p075' 'p081'] 26


(array(['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9'],
       dtype=object),
 array([2489, 2267, 2317, 2346, 2326, 2312, 2325, 2002, 1911, 2129],
       dtype=int64))

In [33]:
y = df["classname"]
X = df.drop(columns=["classname"]).copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train

,subject,img
14217,p049,img_37749.jpg
8980,p026,img_2015.jpg
2129,p014,img_19331.jpg
2414,p014,img_45468.jpg
2794,p015,img_6903.jpg
...,...,...
11964,p045,img_38722.jpg
21575,p075,img_85062.jpg
5390,p021,img_4853.jpg
860,p012,img_74587.jpg


In [11]:
val_img = set(X_test["img"])

AttributeError: 'set' object has no attribute 'head'

In [15]:
# import required module
import os
import shutil
# assign directory
directory_train = '../state-farm-distracted-driver-detection/imgs/train/'
dest_train_c0 = 'src/train/c0'
dest_train_c1 = 'src/train/c1'
directory_test = '../state-farm-distracted-driver-detection/imgs/test/'
dest_val_c0 = 'src/val/c0'
dest_val_c1 = 'src/val/c1'

from distutils.dir_util import copy_tree
""" 
#copy c0 class
copy_tree(directory_train+os.listdir(directory_train)[0], dest_train_c0)

# copy other classes into a folder c1
for folder in os.listdir(directory_train)[1:]:
      #shutil.copy(folder, dest_train_c1)
      copy_tree(directory_train+folder, dest_train_c1)
"""

for file in os.listdir(dest_train_c0):
    if file in val_img:
        shutil.move(os.path.join(dest_train_c0,file), dest_val_c0)

for file in os.listdir(dest_train_c1):
    if file in val_img:
        shutil.move(os.path.join(dest_train_c1,file), dest_val_c1) 
    

In [18]:
# load resnet18:
from torchvision.models import ResNet18_Weights
resnet18 = torchvision.models.resnet18(weights=ResNet18_Weights.DEFAULT)

# Freeze all the pre-trained layers
for param in resnet18.parameters():
    param.requires_grad = False

# Modify the last layer of the model
num_classes = 2 # we start with binary classification
resnet18.fc = torch.nn.Linear(resnet18.fc.in_features, num_classes)

from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms

# Define the transformations to apply to the images
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the train and validation datasets
train_dataset = ImageFolder('src/train', transform=transform)
val_dataset = ImageFolder('src/val', transform=transform)

# Define the loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(resnet18.fc.parameters(), lr=0.001, momentum=0.9)

In [19]:
from torch.utils.data import DataLoader

# Create data loaders for the train and validation datasets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    # Train the model for the specified number of epochs
    for epoch in tqdm(range(num_epochs)):
        # Set the model to train mode
        model.train()

        # Initialize the running loss and accuracy
        running_loss = 0.0
        running_corrects = 0

        # Iterate over the batches of the train loader
        for inputs, labels in train_loader:
            # Move the inputs and labels to the device
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Zero the optimizer gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            # Backward pass and optimizer step
            loss.backward()
            optimizer.step()

            # Update the running loss and accuracy
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        # Calculate the train loss and accuracy
        train_loss = running_loss / len(train_dataset)
        train_acc = running_corrects.double() / len(train_dataset)

        # Set the model to evaluation mode
        model.eval()

        # Initialize the running loss and accuracy
        running_loss = 0.0
        running_corrects = 0

        # Iterate over the batches of the validation loader
        with torch.no_grad():
            for inputs, labels in val_loader:
                # Move the inputs and labels to the device
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Forward pass
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                # Update the running loss and accuracy
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

        # Calculate the validation loss and accuracy
        val_loss = running_loss / len(val_dataset)
        val_acc = running_corrects.double() / len(val_dataset)

        # Print the epoch results
        print('Epoch [{}/{}], train loss: {:.4f}, train acc: {:.4f}, val loss: {:.4f}, val acc: {:.4f}'
              .format(epoch+1, num_epochs, train_loss, train_acc, val_loss, val_acc))


In [24]:
model = resnet18

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [26]:
# Fine-tune the last layer for a few epochs
optimizer = torch.optim.SGD(model.fc.parameters(), lr=0.01, momentum=0.9)
train(model, train_loader, val_loader, criterion, optimizer, num_epochs=5)

Epoch [1/5], train loss: 0.1393, train acc: 0.9501, val loss: 0.0917, val acc: 0.9708


KeyboardInterrupt: 

In [ ]:
# Unfreeze all the layers and fine-tune the entire network for a few more epochs
for param in model.parameters():
    param.requires_grad = True
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
train(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)